In [1]:
import polars as pl

In [2]:
!ls ../data | grep hh_

hh_recsys_sample.pq
hh_recsys_test_hh.pq
hh_recsys_train_hh.pq
hh_recsys_vacancies.pq


In [4]:
compact_log = pl.concat([
    pl.read_parquet('../data/hh_recsys_train_hh.pq'),
    pl.read_parquet('../data/hh_recsys_test_hh.pq'),
])
compact_log.head(2)

user_id,session_id,vacancy_id,action_type,action_dt
str,str,list[str],list[i64],list[datetime[ns]]
"""u_332060""","""s_28301374""","[""v_2571684"", ""v_488179"", … ""v_2633899""]","[2, 2, … 2]","[2023-11-01 00:40:58.105, 2023-11-01 00:58:13.091, … 2023-11-01 01:35:54.456]"
"""u_1057881""","""s_33868982""","[""v_665861""]",[2],[2023-11-01 00:23:51.452]


In [5]:
log = compact_log.select(
    'vacancy_id',
    'action_dt',
).explode(
    'vacancy_id',
    'action_dt',
)
log.head(2)

vacancy_id,action_dt
str,datetime[ns]
"""v_2571684""",2023-11-01 00:40:58.105
"""v_488179""",2023-11-01 00:58:13.091


In [42]:
log['action_dt'].cast(pl.Date).unique().shape

(21,)

In [51]:
(20/21)**95

0.009705470376501574

In [17]:
last_log_dt = log['action_dt'].cast(pl.Date).max()
last_log_dt

datetime.date(2023, 11, 21)

In [27]:
n_action = log.group_by(
    'vacancy_id'
).agg(
    pl.col('action_dt').cast(pl.Date).max(),
    pl.count().alias('n_action'),
).sort('n_action', descending=True)

n_action.head(10)

vacancy_id,action_dt,n_action
str,date,u32
"""v_460169""",2023-11-21,13746
"""v_964765""",2023-11-21,6798
"""v_1840884""",2023-11-21,6687
"""v_2691293""",2023-11-16,6622
"""v_1481785""",2023-11-10,5779
"""v_1507795""",2023-11-21,5259
"""v_2337447""",2023-11-21,4540
"""v_745592""",2023-11-08,4537
"""v_690752""",2023-11-13,4441


In [52]:
deleted = n_action.filter(
    (
        pl.col('n_action') > 95
    ) & (
        pl.col('action_dt') != last_log_dt
    ),
)
deleted.head(5)

vacancy_id,action_dt,n_action
str,date,u32
"""v_2691293""",2023-11-16,6622
"""v_1481785""",2023-11-10,5779
"""v_745592""",2023-11-08,4537
"""v_690752""",2023-11-13,4441
"""v_1455671""",2023-11-19,3946


In [53]:
cp = pl.read_parquet('../data/catboost_predictions_v3.pq').explode('predictions').select(
    pl.exclude('predictions'),
    pl.col('predictions').alias('vacancy_id'),
)
cp.head()

user_id,vacancy_id
str,str
"""u_775363""","""v_622811"""
"""u_775363""","""v_2668345"""
"""u_775363""","""v_390512"""
"""u_775363""","""v_2530865"""
"""u_775363""","""v_1783814"""


In [57]:
cp.join(
    deleted,
    on='vacancy_id',
).group_by(
    'user_id'
).count().max()

user_id,count
str,u32
"""u_999991""",66


In [55]:
cp.shape

(8318900, 2)

In [56]:
1389875 / 8318900

0.16707437281371335